# [컴페티션] 머신러닝 성능 극대화
- 대회 주제 : Binary Classification with a Bank Churn Dataset
- 대회 링크 : https://www.kaggle.com/competitions/playground-series-s4e1
- 평가 항목 : ROC Curve
- 제출일 : 2024년 12월 10일 화요일 7교시
- 평가 항목 점수 : 0.8945/1
- 수강생 성함 : 송현서

In [17]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [18]:
RAND_VAL=27
num_folds=2 ## Number of folds
n_est=1000 ## Number of estimators

In [19]:
df_train = pd.read_csv('data/train.csv')

In [20]:
df_test = pd.read_csv('data/test.csv')
df_test_ov = df_test.copy()

In [21]:
df_orig=pd.read_csv("data/Churn_Modelling.csv")
df_orig=df_orig.rename(columns={'RowNumber':'id'})

In [22]:
# 진짜 의미가 있는 숫자형 데이터만 MinMaxScaler를 사용함.
# MinMaxScaler와 같은 작동방식으로 사용한다.

scale_cols = ['Age','CreditScore', 'Balance','EstimatedSalary']

for c in scale_cols:
    min_value = df_train[c].min()
    max_value = df_train[c].max()
    df_train[c+"_scaled"] = (df_train[c] - min_value) / (max_value - min_value)
    df_test[c+"_scaled"] = (df_test[c] - min_value) / (max_value - min_value)

In [23]:
# train과 test를 합친 다음 인덱스 제거
# 이후 주요 통계 지표 확인

df_all = pd.concat([df_train,df_test]).reset_index(drop=True)
aggs = {
    'Age': ['min','max', 'mean'],       
    'Balance': ['min','max', 'mean','sum'],
    'NumOfProducts': ['mean','sum'],
    'IsActiveMember': ['min','max', 'mean','sum'],
    'CreditScore': ['min','max', 'mean'],
    'EstimatedSalary': ['min','max', 'mean','sum'],
    'id': 'count',
}
df_grps=df_all.groupby(['CustomerId', 'Surname', 'Geography', 'Gender']).agg(aggs).reset_index()
df_grps.columns = list(map(''.join, df_grps.columns.values))
print(len(df_grps))

235258


In [24]:
aggs = {   
    'Balance': ['min','max', 'mean','sum'],
    'NumOfProducts': ['mean','sum'],
    'IsActiveMember': ['min','max', 'mean','sum'],
    'CreditScore': ['min','max', 'mean'],
    'EstimatedSalary': ['min','max', 'mean','sum'],
    'id': 'count',
}
df_grps1=df_all.groupby(['CustomerId', 'Surname', 'Age', 'Gender']).agg(aggs).reset_index()
df_grps1.columns = list(map('grps1'.join, df_grps1.columns.values))
print(len(df_grps1))
df_grps1=df_grps1.rename(columns={'CustomerIdgrps1':'CustomerId','Surnamegrps1':'Surname',
                                 'Agegrps1':'Age','Gendergrps1':'Gender'})

252537


In [25]:
exitGrpBy=['CustomerId', 'Surname',  'Gender','Geography','EstimatedSalary']

exitSrtBy=['CustomerId', 'Surname',  'Gender','Geography',
       'Age', 'Tenure']
##
df_all_Exits=df_all.copy()
df_all_Exits['Exited']=df_all_Exits['Exited'].fillna(-1)
df_all_Exits=df_all_Exits.sort_values(exitSrtBy)
df_all_Exits['Exit_lag1']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(1)
df_all_Exits['Exit_lag2']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(2)
df_all_Exits['Exit_lag3']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(3)

df_all_Exits['Exit_lead1']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(-1)
df_all_Exits['Exit_lead2']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(-2)
df_all_Exits['Exit_lead3']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(-3)

df_all_Exits['Balance_lag_diff1']=df_all_Exits['Balance'].shift(1)
df_all_Exits['Balance_lead_diff1']=df_all_Exits['Balance'].shift(-1)

df_all_Exits=df_all_Exits[['id','Exit_lag1','Exit_lag2','Exit_lag3',
                         'Exit_lead1','Exit_lead2','Exit_lead3',
                          'Balance_lag_diff1','Balance_lead_diff1']]
df_all_Exits=df_all_Exits.fillna(-1).astype('int')
df_all_Exits

,id,Exit_lag1,Exit_lag2,Exit_lag3,Exit_lead1,Exit_lead2,Exit_lead3,Balance_lag_diff1,Balance_lead_diff1
87277,87277,-1,-1,-1,-1,-1,-1,-1,107209
140882,140882,-1,-1,-1,-1,-1,-1,0,161993
160702,160702,-1,-1,-1,-1,-1,-1,107209,161993
259273,259273,-1,-1,-1,-1,-1,-1,161993,0
106193,106193,-1,-1,-1,-1,-1,-1,161993,0
...,...,...,...,...,...,...,...,...,...
189759,189759,-1,-1,-1,-1,-1,-1,0,0
124621,124621,-1,-1,-1,-1,-1,-1,0,135277
97684,97684,-1,-1,-1,-1,-1,-1,0,144591
46972,46972,-1,-1,-1,-1,-1,-1,135277,0


In [26]:
def getGrpsIndv(df_orig,df_train,df_test,grpCols,nm):
    grpBy=[]
    for c in grpCols:
        for i in grpCols:
            if c!=i:
                grpBy=[c,i]
                grpBy=[c,i]
                df_tmp=df_orig.groupby(grpBy).agg({'id':'count','Exited':{'mean'}}).reset_index()
                df_tmp.columns=list(map(''.join, (list(df_tmp.columns.values))))
                sepCols=df_tmp.columns.drop(grpBy)+nm+'_ind_'+c+'-'+i
                df_tmp.columns=list(grpBy)+list(sepCols)
                #
                df_train=df_train.merge(df_tmp,how='left')
                df_test=df_test.merge(df_tmp,how='left')
                
                df_train[sepCols]=df_train[sepCols].fillna(0)
                df_test[sepCols]=df_test[sepCols].fillna(0)

                df_train[sepCols]=df_train[sepCols].astype('int')
                df_test[sepCols]=df_test[sepCols].astype('int')
    return df_train,df_test

In [27]:
def getGrps(df_orig,df_train,df_test,grpCols,nm):
    grpBy=[]
    for c in grpCols:
        grpBy.append(c)
        df_tmp=df_orig.groupby(grpBy).agg({'id':'count','Exited':{'sum'}}).reset_index()
        df_tmp.columns=list(map(''.join, (list(df_tmp.columns.values))))
        sepCols=df_tmp.columns.drop(grpBy)+nm+'_grps_'+c
        df_tmp.columns=list(grpBy)+list(sepCols)
                #
        df_train=df_train.merge(df_tmp,how='left')
        df_test=df_test.merge(df_tmp,how='left')
        
        df_train[sepCols]=df_train[sepCols].fillna(0)
        df_test[sepCols]=df_test[sepCols].fillna(0)
        
        df_train[sepCols]=df_train[sepCols].astype('int')
        df_test[sepCols]=df_test[sepCols].astype('int')
    return df_train,df_test

In [28]:
grpCols=['CustomerId', 'Surname', 'Geography', 'Gender', 'Age', 'Tenure', 'CreditScore', 
         'NumOfProducts','HasCrCard', 
         'IsActiveMember' ,'EstimatedSalary','Balance']
df_train,df_test=getGrps(df_orig,df_train,df_test,grpCols,'Orig_groups')
df_train,df_test=getGrpsIndv(df_orig,df_train,df_test,grpCols,'Orig_ind')

## Feature Engineering

In [29]:
def getFeats(df):
    
    df['IsSenior'] = df['Age'].apply(lambda x: 1 if x >= 60 else 0)
    df['IsActive_by_CreditCard'] = df['HasCrCard'] * df['IsActiveMember']
    df['Products_Per_Tenure'] =  df['Tenure'] / df['NumOfProducts']
    df['AgeCat'] = np.round(df.Age/20).astype('int').astype('category')
    df['Sur_Geo_Gend_Sal'] = df['Surname']+df['Geography']+df['Gender']+np.round(df.EstimatedSalary).astype('str')
    
    df = df.merge(df_grps,how='left',on=['CustomerId', 'Surname', 'Geography', 'Gender'])
    df = df.merge(df_grps1,how='left',on=['CustomerId', 'Surname', 'Age', 'Gender'])
    df = df.merge(df_all_Exits,how='left')
    return df

In [30]:
df_train = getFeats(df_train)
df_test = getFeats(df_test)
##
feat_cols=df_train.columns.drop(['id','Exited'])
feat_cols=feat_cols.drop(scale_cols)
print("Number of Features:",len(feat_cols))
print(feat_cols)
df_train.head()

Number of Features: 352
Index(['CustomerId', 'Surname', 'Geography', 'Gender', 'Tenure',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Age_scaled',
       'CreditScore_scaled',
       ...
       'EstimatedSalarygrps1sum', 'idgrps1count', 'Exit_lag1', 'Exit_lag2',
       'Exit_lag3', 'Exit_lead1', 'Exit_lead2', 'Exit_lead3',
       'Balance_lag_diff1', 'Balance_lead_diff1'],
      dtype='object', length=352)


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,...,EstimatedSalarygrps1sum,idgrps1count,Exit_lag1,Exit_lag2,Exit_lag3,Exit_lead1,Exit_lead2,Exit_lead3,Balance_lag_diff1,Balance_lead_diff1
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,...,181449.97,1,-1,-1,-1,-1,-1,-1,0,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,...,49503.50,1,-1,-1,-1,-1,-1,-1,115587,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,...,184866.69,1,-1,-1,-1,-1,-1,-1,0,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,...,84560.88,1,-1,-1,-1,-1,-1,-1,0,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,...,15068.83,1,-1,-1,-1,-1,-1,-1,135759,142084


In [31]:
X=df_train[feat_cols]
y=df_train['Exited']
##
cat_features = np.where(X.dtypes != np.float64)[0]
cat_features

array([  0,   1,   2,   3,   4,   5,  12,  13,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 18

## Training

In [32]:
folds = StratifiedKFold(n_splits=num_folds,random_state=RAND_VAL,shuffle=True)
test_preds = np.empty((num_folds, len(df_test)))
auc_vals=[]

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[valid_idx], y.iloc[valid_idx]
    
    train_pool = Pool(X_train, y_train,cat_features=cat_features)
    val_pool = Pool(X_val, y_val,cat_features=cat_features)
    
    clf = CatBoostClassifier(
    eval_metric='AUC',
    task_type='CPU',
    learning_rate=0.02,
    iterations=n_est)
    clf.fit(train_pool, eval_set=val_pool,verbose=300)
    
    y_pred_val = clf.predict_proba(X_val[feat_cols])[:,1]
    auc_val = roc_auc_score(y_val, y_pred_val)
    print("AUC for fold ",n_fold,": ",auc_val)
    auc_vals.append(auc_val)
    
    y_pred_test = clf.predict_proba(df_test[feat_cols])[:,1]
    test_preds[n_fold, :] = y_pred_test
    print("----------------")

0:	test: 0.8528796	best: 0.8528796 (0)	total: 1.59s	remaining: 26m 29s
300:	test: 0.9004386	best: 0.9004386 (300)	total: 19m 53s	remaining: 46m 10s
600:	test: 0.9023322	best: 0.9023322 (600)	total: 1h 33m 26s	remaining: 1h 2m 1s


## Evaluation

In [ ]:
"Mean AUC: ",np.mean(auc_vals)

## Feature Importance

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(train_pool)
shap.summary_plot(shap_values, X_train, plot_type="bar")

## Prediction and Submission

In [ ]:
join_cols=list(df_orig.columns.drop(['Exited']))
df_orig.rename(columns={'Exited':'Exited_Orig'},inplace=True)
df_orig['Exited_Orig']=df_orig['Exited_Orig'].map({0:1,1:0})
df_test_ov=df_test_ov.merge(df_orig,on=join_cols,how='left')[['id','Exited_Orig']].fillna(-1)
df_test_ov.head()

In [ ]:
y_pred = test_preds.mean(axis=0)
df_sub = df_test_ov[['id','Exited_Orig']]
df_sub['Exited'] = np.where(df_sub.Exited_Orig==-1,y_pred,df_sub.Exited_Orig)
df_sub.drop('Exited_Orig',axis=1,inplace=True)
df_sub.head()

In [ ]:
df_sub.to_csv("submission.csv",index=False)

In [ ]:
df_sub.hist(column='Exited', bins=20, range=[0,1],figsize=(12,6))
plt.show()